-> Unsupervised meaning we have input features but we don't have the labels

eg: Suppose we want to create a system that will take a few pictures of each item on a manufacturing production line and detect which items are defective.

-> Unsupervised Learning tasks

1. Dimensionality reduction

Clustering for dimesnionality reduction - Once a dataset has been clustered, its usually possible to measure each instance's affinity with each cluster. affinity is any measure of how well an instance fits into a cluster. Each instance's feature vector x can then be replaced with the vector of its cluster affinities. if there are k clusters then this vector is k-dimensional. New vector is typically much lower dimensional than the original feature vector, but it can preserve enough information for further processing.

2. Clustering

Goal is to group similar instances together into clusters. Clustering is a great tool for data analysis, customer segmentation, recommender systems, search engines (To search for images similar to a reference image. First apply a clustering algorithm to all the images in the database. SImilar images will form a cluster), image segmentation, semi-supervised learning (If we have only few labels, do clustering and propagate the labels to all the instances in the same cluster), dimensionality reduction etc.

eg: k-means,DBSCAN

3. Anomaly (or Outlier) detection

Objective is to learn what normal data looks like and then use that to detect abnormal instances. These instances are called anomalies or outliers while normal instances are called inliers. Useful in fraud detection, detecting defective products in manufacturing, identifying new trends in time series, removing outliers from dataset before training another model which can significantly improve the performance of the resulting model. Using clustering, any instance with low affinity to all the clusters is likely to be an anomaly.

4. Density estimation

Task of estimating probability density function (PDF) of the random process that generated the dataset. Commonly used for anomaly detection. Instances located in very low density regions are likely to be anomalies. Useful for data analysis and visualization.

# k-means (Lloyd-Forgy algorithm)

-> One of the fastest clustering algorithms

-> Start by placing the centroids randomly eg: by picking k instances at random from the dataset and using their locations as centroids. Then label the instances - assigning to the cluster whose centroid is the closest , update the centroids - by computing the mean of the instances in that cluster, and so on until the centroid stops moving. Algorithm is guaranteed to converge because the mean squared distance between the instances and their closest centroids can only go down at each step and since it cannot be negative, its guaranteed to converge.

-> Important to scale input featuresbefore running k-means otherwise clusters may be very stretched and kmeans will perform poorly.

-> Hard clustering - Each instance assigned to single cluster

-> Soft clustering - Give each instance a score per cluster. Score - distance from centroid or similarity score like Gaussian radial basis function

-> Centrid initialization methods

1. If we know approx where the centroids should be

good_init=np.array([[-3,3],[-3,2],[-3,1],[-1,2],[0,2]])

kmeans=KMeans(n_clusters=5,init=good_init,n_init=1,random_state=42)

kmeans.fit(X)

2. Run algorithm multiple times with different random initializations and keep the best solution. Number of random initializations is controlled by n_init hyperparameter. By default its 10. To determine the best solution, model's inertia which is the sum of the squared distances between the instances and their closest centroids is used as performance metric (WCSS - Within cluster sum of squares. Its a measure of variability of data points within each cluster).

WCSS = sigma i=1 to k sigma j=1 to mi (Xij-Ci)^2

-> Finding optimal number of clusters

When k is too small => separate clusters get merged.

When k is too large => Some clusters get chopped into multiple pieces.

As we increase k inertia will be reduced. More clusters there are closest each instance will be to its closest centroid so less inertia. So its not a good performance metric in choosing number of clusters. Plot inertia as a function of k. Curve will have an elbow point.Inertia drops very quickly as we increase k till thatb point after that inertia decreases very slowly.

Silhouette score - Mean Silhouette coefficient over all the instances.

Silhouette coefficient = (b-a)/max(a,b) where a=mean intra cluster distance ie mean distances to other instances in the same cluster , b=mean nearest cluster distance ie mean distance to the instances of the nest closest cluster excluding instance's own cluster. It can vary from -1 to +1.

+1 => Instance is well inside its own cluster and far from other clusters.

0=> Its close to a cluster boundary

-1 => It may have been assigned to wrong cluster.

Silhouette diagram - Plot every instance's silhouette coefficient sorted by the clusters they are assigned to and by the value of the coefficient. Each diagram contains one knife shape per cluster. Shape's height indicates number of instances in that cluster. Width represents sorted silhouette coefficients of the instances in the cluster. Wider is better.

-> Drawbacks of k-means

1. May get bad results because of random initialization of the centroids

2. May not give the best results for data where the clusters are of varying size or density.

3. Number of clusters (k) is not defined

-> Accelerated k-means

On large datasets with many clusters, algorithm can be accelerated by avoiding many unnecessary distance calculations. Exploiting triangle inequality and by keeping track of lower and upper bounds for distances between instances and centroids. algorithm='elkan'

-> Mini batch k-means

Instead of using full datset, using mini batches moving the centroids slightly at each iteration. Speeds up the algorithm Makes it possible to cluster huge datasets that do not fit in memory. It will have higher inertia than k-means but faster.

from sklearn.cluster import MiniBatchKMeans

minibatch_kmeans=MiniBatchKMeans(n_Clusters=5,random_state=42)

minibatch_kmeans.fit(X)

# K-means++

Sklearn by default uses k-means++.
Initialization step tends to select centroids that are distant from one another. This makes k=means algo much less likely to converge to suboptimal solution.

1. Take one centroid randomly from dataset.

2. Take new centroid choosing an instance xi with probability D(xi)^2/sigma j=1 to m D(xj)^2 . Instances far away from already chosen centroids are much more likely to be selected.

3. Repeat 2 until k centroids have been chosen.

-> Limitations

Its necessary to run algorithm multiple times to avaoid suboptimal solutions plus we need to specify number of clusters. When clusters have varying sizes different densities nonspherical shapes, kmeans doesnt behave very well.

# Hierarchical clustering

Hierarchy of clusters is developed in the form of a tree and this tree shaped structure is known as the dendogram.

1. Compute the distance (Euclidean, Manhattan, Hamming Distance) between all the data points.

2. Merge 2 points having minimum distance between them.

3. Treat the subclusters as points and repeat 2 util one cluster remains.

4. Finally develop the dendogram to divide the clusters as per the problem.

Common type of linkages between 2 clusters are

-> Complete Linkage - Maximum distance b/w 2 farthest points

-> Single Linkage - Minimum distance b/w 2 closest points

-> Centroid Linkage - Distance b/w centroids

-> Ward's distance/Linkage - b/w 2 clusters c1 and c2 is

W(c1+c2) = WCSS(c1+c2)-WCSS(c1)-WCSS(C2)

No need to pre-decide k value. Good for data visualization providing hierarchical relation b/w the clusters.

This method is sensitive to the choice of linkage function. It can be used only for analysis not for making inferences. If we get a new data point we need to rerun the entire algorithm with all the existing points and reanalyze.


In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
import numpy as np

blob_centers = np.array([[ 0.2,  2.3], [-1.5 ,  2.3], [-2.8,  1.8],
                         [-2.8,  2.8], [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers, cluster_std=blob_std,
                  random_state=7)

In [ ]:
k=5
kmeans=KMeans(n_clusters=k,random_state=42)
y_pred=kmeans.fit_predict(X)
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([4, 0, 1, ..., 2, 1, 0], dtype=int32)

In [ ]:
kmeans.cluster_centers_

array([[-2.80389616,  1.80117999],
       [ 0.20876306,  2.25551336],
       [-2.79290307,  2.79641063],
       [-1.46679593,  2.28585348],
       [-2.80037642,  1.30082566]])

In [ ]:
X_new=np.array([[0,2],[3,2],[-3,3],[-3,2.5]])
kmeans.predict(X_new)

array([1, 1, 2, 2], dtype=int32)

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X,kmeans.labels_)

0.655517642572828

# DBSCAN (Density based spatial clustering of applications with noise)

1. For each instance, the algorithm counts how many instances are located within a small distance epsilon from it. This region is called the instance's epsilon-neighbourhood.

2. If an instance has at least min_samples instances in its epsilon-neighbourhood then its considered a core instance. Core instances are those that are located in dense regions.

3. Long sequence of core instances forms a single cluster. All instances in the neighborhood of a core instance belong to the same cluster.

4. Any instance that is not a core instance and doesnt have one in its neighborhood is considered anomaly.

Core point - Point that has atleast minPts points within distance eps from itself.

Border point - Point that has atleast one Core point at a distance eps

Noise point - Neither a core nor a border. And it has < minPts points within distance eps from itself.

-> DBSCAN doesnt have predict(). It cannot predict which cluster a new instance belongs to. We can use different classification algos for that.

-> Powerful algo capable of identifying any no of clusters of any shape.Robust to outliers. Doesn't work well with high dimensional data. DBSCAN struggles with clusters of similar density.

-> 2 hyperparameters - epsilon,min_samples. One way to decide initial value of eps is by plotting histogram of distance between each point. We may get 2 peaks. Initial value of eps may be one of the distance values between these 2 peaks.

1. Pick a random point from the dataset and if its a core point, assign it to cluster 1 and mark it as visited.

2. If not, point is marked as noise. Assign all the points within the neighborhood of the initial point to cluster 1. If these new points are core points assign its neighborhood points also to cluster1.

3. Next randomly choose another unvisited point and repeat step 1 with it.

4. Stop when all points are visited.

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons

X,y=make_moons(n_samples=1000,noise=0.05)
dbscan=DBSCAN(eps=0.05,min_samples=5)
dbscan.fit(X)

DBSCAN(eps=0.05)

In [ ]:
dbscan.labels_     ## labels of all instances
dbscan.core_sample_indices_  ## indices of core instances
dbscan.components_     ## core instances

array([[-0.67588346,  0.79528251],
       [ 1.95994091,  0.33687059],
       [ 1.99458094,  0.17977172],
       ...,
       [-1.0212242 ,  0.1376901 ],
       [ 1.94821479,  0.10318642],
       [-0.70707713,  0.65859321]])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier(n_neighbors=50)
knn.fit(dbscan.components_, dbscan.labels_[dbscan.core_sample_indices_])

## trained only on core instances. We could also have chosen to train it on all the instances or all but the anomalies. Choice depends on final task.

KNeighborsClassifier(n_neighbors=50)

In [ ]:
X_new=np.array([[-0.5,0],[0,0.5],[1,-0.1],[2,1]])
knn.predict(X_new)
knn.predict_proba(X_new)

array([[0.78, 0.  , 0.  , 0.22, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ],
       [0.72, 0.  , 0.14, 0.  , 0.14, 0.  , 0.  , 0.  ],
       [0.  , 0.76, 0.  , 0.  , 0.  , 0.16, 0.  , 0.08]])

# Gaussian Mixture Model (GMM)

Its a probabilistic model that assumes that the instances were generated from a mixture of several Gaussian distributions whose paraemters are unknown.All the instances generated from a single Gaussian distribution form a cluster that typically looks like an ellipsoid. Each cluster can have a different ellipsoidal shape,size,density and orientation. It uses soft clustering approach for distributing the points in different clusters.

-> **GMM Algorithm (1D)**

1. Randomly initialize mu and sigma

2. Update mu and sigma

*   Calc prob of each pt belonging to each gaussian
*   Assign all the pts which have the highest prob to their resp gaussians
*   Compute new mu for each gaussian as the mean of the pts belonging to that gaussian
*   Similarly update the variance
*   After multiple updates, we will have tightly fitting Gaussian disbns representing different clusters.

-> **2D/ND Gaussians**

2 variables x and y. Multivariate disbn is defined by 5 parameters:

mux-mean of x coordinate

muy-mean of y coordinate

sigmax^2-variance in x direction

sigmay^2-variance in y direction

Cov(x,y)-Covariance b/w x and y







# Anomaly detection algorithms

**1. RANSAC**

A dataset with number of points having parameters mu and sigma^2

-> Sample a subset of points (n). We consider this as inliers.

-> Now compute a model that estimates parameters of the sampled points.

-> Score the model which indicates how many points will support the model.

Repeat above steps and select best model. That tells the points which are inliers and outliers.


**2. Elliptic Envelope**

X follows normal distribution being unimodal. Elliptical envelope robustly estimate The parameters mu_vector,Sigma ()covariance matrix

We remove the points that are outliers which are far away from the centroid.

**3. Isolation Forests**

Build many trees. For each tree,

Randomly pick a feature, Randomly threshold that feature, Build each tree until the leaf consists of only one data point.

On an average outliers have lower depth in random trees and inliers have more depth.

**4. Local Outlier Factor**

Based on 2 ideas - KNN, density. Compare the density of a point with its neighbor's density. If the density of a point is less than the density of its neighbors, we flag that point as an outlier. We compute density based on average distance. If average distance between  a point and its K nearest neighbors is large, its more likely that the point will be an outlier. Larger the value of K, more confident are the results.
